In [1]:
import numpy as np
import sys
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

%matplotlib inline
import glob
import warnings
warnings.filterwarnings('ignore')
from pprint import pprint
import importlib

import math
import geopy
from geopy import distance

In [2]:
#download the ECCO grid file

from ecco_download import *

ecco_podaac_download(ShortName="ECCO_L4_GEOMETRY_LLC0090GRID_V4R4",\
                    StartDate="1992-01-01",EndDate="2010-12-31",download_root_dir=None,\
                    n_workers=6,force_redownload=False)

created download directory /home/m267zhou/Downloads/ECCO_V4r4_PODAAC/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4
{'ShortName': 'ECCO_L4_GEOMETRY_LLC0090GRID_V4R4', 'temporal': '1992-01-01,2010-12-31'}

Total number of matching granules: 1

GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc already exists, and force=False, not re-downloading
DL Progress: 100%|########################| 1/1 [00:00<00:00, 10645.44it/s]

total downloaded: 0.0 Mb
avg download speed: 0.0 Mb/s


In [3]:
# download file (granule) containing 2000 velocities,
# to default path ~/Downloads/ECCO_V4r4_PODAAC/
vol_monthly_shortname = "ECCO_L4_OCEAN_3D_VOLUME_FLUX_LLC0090GRID_MONTHLY_V4R4"
ecco_podaac_download(ShortName=vol_monthly_shortname,\
                    StartDate="1992-01-01",EndDate="2017-12-31",download_root_dir=None,\
                    n_workers=6,force_redownload=False)

created download directory /home/m267zhou/Downloads/ECCO_V4r4_PODAAC/ECCO_L4_OCEAN_3D_VOLUME_FLUX_LLC0090GRID_MONTHLY_V4R4
{'ShortName': 'ECCO_L4_OCEAN_3D_VOLUME_FLUX_LLC0090GRID_MONTHLY_V4R4', 'temporal': '1992-01-01,2017-12-31'}

Total number of matching granules: 312

OCEAN_3D_VOLUME_FLUX_mon_mean_1992-02_ECCO_V4r4_native_llc0090.nc already exists, and force=False, not re-downloading

OCEAN_3D_VOLUME_FLUX_mon_mean_1992-01_ECCO_V4r4_native_llc0090.nc already exists, and force=False, not re-downloading

OCEAN_3D_VOLUME_FLUX_mon_mean_1992-03_ECCO_V4r4_native_llc0090.nc already exists, and force=False, not re-downloading

OCEAN_3D_VOLUME_FLUX_mon_mean_1992-04_ECCO_V4r4_native_llc0090.nc already exists, and force=False, not re-downloading

OCEAN_3D_VOLUME_FLUX_mon_mean_1992-05_ECCO_V4r4_native_llc0090.nc already exists, and force=False, not re-downloading

OCEAN_3D_VOLUME_FLUX_mon_mean_1992-06_ECCO_V4r4_native_llc0090.nc already exists, and force=False, not re-downloading

OCEAN_3D_VOLUM

In [4]:
## Import the ecco_v4_py library into Python
## =========================================
##    If ecco_v4_py is not installed in your local Python library, 
##    tell Python where to find it.  The example below adds
##    ecco_v4_py to the user's path if it is stored in the folder
##    ECCOv4-py under the user's home directory

from os.path import join,expanduser
user_home_dir = expanduser('~')

sys.path.append(join(user_home_dir,'ECCOv4-py'))

import ecco_v4_py as ecco

In [5]:
## Set top-level file directory for the ECCO NetCDF files
## =================================================================

## currently set to ~/Downloads/ECCO_V4r4_PODAAC, 
## the default if ecco_podaac_download was used to download dataset granules
ECCO_dir = join(user_home_dir,'Downloads','ECCO_V4r4_PODAAC')

In [6]:
## Load the model grid
ecco_grid = xr.open_dataset(glob.glob(join(ECCO_dir,'*GEOMETRY*','*.nc'))[0])

In [7]:
#To store zonal integral of meridional velovity in 2D arrays (integrate over all longitudes) at latitude of 10 degree over all years 
#in the Atlantic Ocean.
#Dimension is depth by months.
zonal_integral_for_VVELMASS_degree10 = np.zeros((50, 26*12))

In [8]:
## Load vector fields
ecco_vars1 = xr.open_mfdataset(join(ECCO_dir,'*OCEAN*VOLUME*MONTHLY*','*_1992-*.nc'))
ecco_ds = []

## Merge the ecco_grid with the ecco_vars to make the ecco_ds
ecco_ds = xr.merge((ecco_grid,ecco_vars1)).load()

In [9]:
# Exclude pacific ocean
xfld = ecco_ds.UVELMASS.isel(k=0, time=0)
yfld = ecco_ds.VVELMASS.isel(k=0, time=0)
# Compute the zonal and meridional vector components of UVEL and VVEL
VEL_E, VEL_N  = ecco.vector_calc.UEVNfromUXVY(xfld, yfld, ecco_ds)
valid_lon_for_each_lat = np.zeros(90)
for lat in range(0,90):
    for lon in range(0,90):
        if (math.isnan(VEL_E.isel(tile=10)[lon,lat]) == True):
            valid_lon_for_each_lat[lat] = lon
            break
        else:
            continue

In [10]:
#Compute zonal integral for VVELMASS at 10 degree of latitude over all months
for year in range(0, 26):
    y = 1992 + year
    ecco_vars1 = xr.open_mfdataset(join(ECCO_dir,'*OCEAN*VOLUME*MONTHLY*','*_'+str(y)+'-*.nc'))
    ecco_ds = []
    ## Merge the ecco_grid with the ecco_vars to make the ecco_ds
    ecco_ds = xr.merge((ecco_grid,ecco_vars1)).load()
    
    for depth in range(0,50):
        for t in range(0, 12):
            xfld = ecco_ds.UVELMASS.isel(k=depth, time=t)
            yfld = ecco_ds.VVELMASS.isel(k=depth, time=t)
        # Compute the zonal and meridional vector components of UVEL and VVEL
            VEL_E, VEL_N  = ecco.vector_calc.UEVNfromUXVY(xfld, yfld, ecco_ds) 

            sum = 0
            for lon in range(0,89):
                if ((math.isnan(VEL_N.isel(tile=10)[lon,89]) == True) or (valid_lon_for_each_lat[89] >= lon)):
                    sum = sum
                else:
                    sum = sum + VEL_N.isel(tile=10)[lon,89] * distance.distance((VEL_N.isel(tile=10).YC[lon,89], VEL_N.isel(tile=10).XC[lon,89]), (VEL_N.isel(tile=10).YC[lon+1,89], VEL_N.isel(tile=10).XC[lon+1,89])).m
            for lon in range(0,89):
                if (math.isnan(VEL_N.isel(tile=2)[89-89, lon]) == True):
                    sum = sum
                else:
                    sum = sum + VEL_N.isel(tile=2)[89-89, lon] * distance.distance((VEL_N.isel(tile=2).YC[89-89, lon], VEL_N.isel(tile=2).XC[89-89, lon]), (VEL_N.isel(tile=2).YC[89-89, lon+1], VEL_N.isel(tile=2).XC[89-89, lon+1])).m
        
            zonal_integral_for_VVELMASS_degree10[depth, year*12+t] = sum

In [11]:
#create amoc array
amoc_for_VVELMASS_at_lat10 = np.zeros(26*12)

#get depth array
depth_array = ecco_grid.Z[0:50].values

In [12]:
#compute amoc at 10 degree of latitude over months
for t in range(0,26*12):
    sum = 0
    arr = np.zeros(49)
    for depth in range(0, 49):
        sum = sum + zonal_integral_for_VVELMASS_degree10[depth, t] *  (-(depth_array[depth+1] - depth_array[depth]))
        arr[depth] = sum
    amoc_for_VVELMASS_at_lat10[t] = np.max(arr)

In [13]:
#store month array and amoc array in csv file for later use
arr = [np.arange(0, 26*12, 1), amoc_for_VVELMASS_at_lat10/1000000]
np.savetxt("10.csv", arr, delimiter =",")